# Create a Speech-Enabled Apps with Azure AI Services

### Text Analysis

#### Step1. Build **AI Language** and get the API Keys and Endpoint of it from Azure portal.

In [1]:
LANGUAGE_AI_KEY = 'bf2f2efce0a84c0ab3032afd10e20b97'
LANGUAGE_AI_ENDPOINT = 'https://mslearn-ais-language.cognitiveservices.azure.com/'

#### Step2. Build a Text Analytics Client

In [2]:
!pip install azure-ai-textanalytics==5.3.0
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient

text_analysis_client = TextAnalyticsClient(endpoint=LANGUAGE_AI_ENDPOINT, credential=AzureKeyCredential(LANGUAGE_AI_KEY))

Defaulting to user installation because normal site-packages is not writeable


Step3. Performing Text Analysis

In [3]:
text = 'Welcome to Microsoft Azure, Markov. Nice to see you!'

# detect language
res = text_analysis_client.detect_language(documents=[text])[0]
print('\nLanguage: {}'.format(res.primary_language.name))


Language: English


In [4]:
# evaluate sentiment
res = text_analysis_client.analyze_sentiment(documents=[text])[0]
print("\nSentiment: {}".format(res.sentiment))


Sentiment: positive


In [5]:
# identify key phrases
res = text_analysis_client.extract_key_phrases(documents=[text])[0].key_phrases
if len(res) > 0:
    print("\nKey Phrases:")
    for phrase in res:
        print('\t{}'.format(phrase))


Key Phrases:
	Microsoft Azure
	Markov


In [6]:
# extract entities
res = text_analysis_client.recognize_entities(documents=[text])[0].entities
if len(res) > 0:
    print("\nEntities")
    for entity in res:
        print('\t{} ({})'.format(entity.text, entity.category))


Entities
	Microsoft Azure (Product)
	Markov (Person)


In [7]:
# extract linked entities
res = text_analysis_client.recognize_linked_entities(documents=[text])[0].entities
if len(res) > 0:
    print("\nLinks")
    for linked_entity in res:
        print('\t{} ({})'.format(linked_entity.name, linked_entity.url))


Links
	Microsoft Azure (https://en.wikipedia.org/wiki/Microsoft_Azure)
	OGC Nice (https://en.wikipedia.org/wiki/OGC_Nice)


### Question and Answering

#### Step1. Build AI Search and create your QnA pairs on [Language Studio](https://language.cognitive.azure.com/home).¶

* AI Language ->
* Language Studio ->
* **Custom question answering** ->
* Build an Project named `basic-kb` with **AI Language** and **AI Search** we used.
* upload prebuilt QnA pairs (`./assets/basic-kb_qa_pairs.tsv`) to this project.
* Deploy this knowledge base.
* Click **get prediction URL** and then find the `deploymentName`

#### Step2. Build a Question Answering Client

In [8]:
!pip install azure-ai-language-questionanswering
from azure.core.credentials import AzureKeyCredential
from azure.ai.language.questionanswering import QuestionAnsweringClient
client = QuestionAnsweringClient(endpoint=LANGUAGE_AI_ENDPOINT, credential=AzureKeyCredential(LANGUAGE_AI_KEY))

Defaulting to user installation because normal site-packages is not writeable


#### Step3. Performing Question Answering

In [9]:
user_question = 'Why is my URL(s)/file(s) is not extracting question-answer pairs?'
response = client.get_answers(question=user_question,
                                project_name='basic-kb',
                                deployment_name='production')
for candidate in response.answers:
    print(candidate.answer)
    print("Confidence: {}".format(candidate.confidence))
    print("Source: {}".format(candidate.source))

It's possible that QnA Maker can't auto-extract some question-and-answer (QnA) content from valid FAQ URLs. In such cases, you can paste the QnA content in a .txt file and see if the tool can ingest it. Alternately, you can editorially add content to your knowledge base through the [QnA Maker portal](https://qnamaker.ai).
Confidence: 1.0
Source: https://docs.microsoft.com/azure/cognitive-services/qnamaker/faqs


### Language Understanding

#### Stpe1. Create your intents examples on [Language Studio](https://language.cognitive.azure.com/home).¶

* AI Language ->
* Language Studio ->
* **Conversational language understanding (Only provided by few region.)** -> 
* Build an Project named `Clock` with **AI Language** we used.
* Add the following utterances for intents:

  a. GetTime<br>
    ``` 
    - what's the time    ?
    - what time is i    t?
    - tell me the t   
    ```
  b. GetDay<br>
    ```
    - what day is it?
    - what's the day?
    - what is the day today?
    - what day of the week is it?
    ```
  c. GetDate<br>
    ```
    - what date is it?
    - what's the date?
    - what is the date today?
    - what's today's date?
    ```
* Fine-tune a model for this and Deploy it.

#### Step2. Build a Language Understanding Client

In [10]:
!pip install azure-ai-language-conversations
from azure.core.credentials import AzureKeyCredential
from azure.ai.language.conversations import ConversationAnalysisClient

client = ConversationAnalysisClient(endpoint=LANGUAGE_AI_ENDPOINT, credential=AzureKeyCredential(LANGUAGE_AI_KEY))

Defaulting to user installation because normal site-packages is not writeable


In [11]:
query = "What's the date and day today?"

result = client.analyze_conversation(
    task={"kind": "Conversation",
          "analysisInput": {
              "conversationItem": {"participantId": "1", "id": "1", "modality": "text", "language": "en", "text": query},
          "isLoggingEnabled": False
        },
        "parameters": {"projectName": 'Clock', "deploymentName": 'Clock', "verbose": True}
         }
)
print("ttop intent: {}".format(result["result"]["prediction"]["topIntent"]))
print("tcategory: {}".format(result["result"]["prediction"]["intents"][0]["category"]))
print("tconfidence score: {}\n".format(result["result"]["prediction"]["intents"][0]["confidenceScore"]))

ttop intent: GetDate
tcategory: GetDate
tconfidence score: 0.95870745



### Extract Entities (*Group admin is required)

#### Stpe1. Create your entities on [Language Studio](https://language.cognitive.azure.com/home).¶

#### Step2. Build a Language Understanding Client

In [12]:
!pip install azure-ai-textanalytics==5.3.0
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient

client = TextAnalyticsClient(endpoint=LANGUAGE_AI_ENDPOINT, credential=AzureKeyCredential(LANGUAGE_AI_KEY))

Defaulting to user installation because normal site-packages is not writeable


### Audio-to-Text & Text-to-Audio

#### Step1. Build **AI Speech** and get the API Keys and Endpoint of it from Azure portal.

In [13]:
SPPECH_AI_KEY = 'e60a931a1fe7463b8528229a1a5d268a'
SPEECH_AI_REGION = 'westus'

#### Step2. Create an Speech Client

In [21]:
!pip install azure-cognitiveservices-speech==1.30.0
import azure.cognitiveservices.speech as speech_sdk

audio_config = speech_sdk.AudioConfig(use_default_microphone=True)
speech_config = speech_sdk.SpeechConfig(SPPECH_AI_KEY, SPEECH_AI_REGION)
speech_config.speech_synthesis_voice_name = "en-GB-RyanNeural"

recorder = speech_sdk.SpeechRecognizer(speech_config, audio_config)
synthesizer = speech_sdk.SpeechSynthesizer(speech_config)

Defaulting to user installation because normal site-packages is not writeable


#### Step3. Performing Audio-to-Text by Microphone

In [22]:
input("Press 'Enter' to Start Recording...(ask some question about basic-kb.tsv)")
response = recorder.recognize_once_async().get().text
print('Response:', response)

Press 'Enter' to Start Recording...(ask some question about basic-kb.tsv) 


Response: What time is it?


#### Step4. Performing Text-to-Audio

In [23]:
synthesizer.speak_text_async(response).get()